# OME-Zarr generation

## INPUT PARAMETERS

In [1]:
#generate OME-Zarr
import ome_zarr.reader 
import ome_zarr.scale  
import ome_zarr.writer 
import zarr, skimage, tifffile, os
import numpy as np 
import pandas as pd 

barcodes = pd.read_csv(r'./barcode.csv')
N_Target = barcodes.shape[0]


Tile_SZ = [3200, 5440]              # assuming no changes, global parameters
Col, Row, Z = 3, 1, 6               # pick a well and a Z
Zs     = [1,3,5,7]                  # for quick QC only
Tiles  = [1, 2, 3, 4]
Swaths = [1, 2, 3, 4]
Cycles = [1, 2, 3, 4, 5]
Channels= [1, 2, 3]                 # 3N
Cyc_0, Ch_0 = 0, 0                  # aligned all data to cyc-0, ch-0

Data_Path = "../../data/"                       # raw data 
TMP_Path = f"../col_{Col}_row_{Row}/tmp/"       # intermediate folder
Result_Path = f"../col_{Col}_row_{Row}/result/"  # result folderd

Barcodes_Path = "./barcode.csv"                 # barcodes info
RootF = "Col_ccc_Swath_sss_Tile_00fff_Cyc_0ttt_row_rrr_z_zzz_seq_1.tiff"        #file format
RootFAligned = f"{TMP_Path}c_ccc_ch_kkk.tiff"

conv = False
cyc_conv = {0:0, 2:1, 4:2}
def CTGA2CycCh(barcode):
    barcode = list( barcode )
    nt = len(barcode)
    cyc, ch = 0, 0
    ch = {'C':0, 'T':1, 'G':2, 'A':3}
    for i,bp in enumerate(barcode):
        if bp != 'A': 
            cyc, ch = i, ch[bp]
            break
    return cyc, ch


for idx,arow in barcodes.iterrows():
    target = arow['target']
    cyc, ch = CTGA2CycCh( arow['barcode'])
    if conv:
        cyc = cyc_conv[cyc]
    print( f'{target}\t cyc={cyc}\t ch={ch}')
    

CD20	 cyc=0	 ch=0
CD68	 cyc=0	 ch=1
CD11b	 cyc=2	 ch=0
CD3e	 cyc=2	 ch=1
CD4	 cyc=4	 ch=0
HLADR	 cyc=4	 ch=1
HLAABC	 cyc=4	 ch=2


## CREATE A MASTER FILE

In [15]:
#create a big tiff file no xtalk correction 
dimension_order = 'CYX'

cnt, cnt_max = 0, 10000
imgs = np.zeros( (N_Target, Tile_SZ[0]*len(Tiles), Tile_SZ[1]*len(Swaths)), dtype=np.uint16)
try:
    for idx, arow in barcodes.iterrows():
        target = arow['target']
        cyc, ch= CTGA2CycCh( arow['barcode'] )
        cyc, ch= cyc + 1, ch + 1
        
        f = RootFAligned.replace('ccc', f'{cyc}')
        f = f.replace('kkk', f'{ch}')
        
        imgs[idx,:,:] = skimage.io.imread(f)

        cnt+=1
        if cnt>cnt_max:
            raise StopIteration
except StopIteration:
    pass
output_file = f"../col_{Col}_row_{Row}/result/col_{Col}_row_{Row}_z_{Z}_no_corr.zarr"

## CREATE OME-ZARR

In [18]:
output_file = f"../col_{Col}_row_{Row}/result/col_{Col}_row_{Row}_z_{Z}_no_corr.zarr"

In [25]:
# generate _zarr file 
#scale down by 2
# dimension_order 'cYX'

import os 
appendix = 1
f = output_file
while os.path.exists( f ):
    f = output_file.replace('.zarr', f'_{appendix}.zarr')
    appendix += 1

scaler = ome_zarr.scale.Scaler()
mip = scaler.local_mean(imgs)

loc   = ome_zarr.io.parse_url(f, mode="w")
group = zarr.group( loc.store )
axes= ("c", "y", "x")
# ome_zarr.writer.write_image(
#                             image = img,
#                             group = group,
#                             axes  = axes)
ome_zarr.writer.write_multiscale(
                                 pyramid=mip,
                                 group=group,
                                 axes=axes)
napari_path = f

## NAPARI ANNOTATIONS

In [2]:
colormaps = ['blue', 'gray', 'yellow', 'green', 'magenta', 'red',
            'bop blue',
            'bop orange',
            'bop purple',
            'cyan',
            'magenta']
colormaps.reverse()
n_colors = len( colormaps )
n_repeat = int( np.round( N_Target/n_colors ) + 1 )
cmaps = colormaps * n_repeat
cmaps = cmaps[:N_Target]


# napari_path = r"../col_3_row_1/result/col_3_row_1_z_6_no_corr.zarr"
import napari
viewer = napari.Viewer()
viewer.open(napari_path)

for idx,arow in barcodes.iterrows():
    # viewer.layers[idx].colormap = cmaps[idx]
    viewer.layers[idx].name = str( barcodes['target'][idx])
